# Project Summary

## Overall Approach

This project first preprocess the raw data, including steps of checking null values, compute relevant features and aggregating the collected features. 

这里稍微写一下preprocess做了那些feature

Then, according to each type of products provided by the merchants, we separted them into 5 main categories: 'Technology and Equipment, Luxury, Entertainment, Services, Stationeries'.

Below is the weekly income of Australian by postcodes, and we noticed that it might have some relationship with the consumers purchasing power.  

In [ ]:
import webbrowser
webbrowser.open('../plots/income_weekly.html', new=2)

Therefore, the postcodes with the most number of transaction of each merchant is taken as a feature for further analysis.

## The Ranking System

Our final ranking system uses the follow 3 features: Monthly income, Monthly profit growth, and fraud probability.
With the formula below:

$ EstimatedProfit = (MonthlyProfit + 12 * ProfitGrowth) * (1 - FraudProbability) $

### Monthly Income

### Monthly Profit Growth

### Fraud Probability

## Ranking System Results